In [24]:
# encoding: utf-8

# https://stackoverflow.com/questions/26567843/reading-the-mail-content-of-an-mbox-file-using-python-mailbox
# https://www.debuggex.com/cheatsheet/regex/python

import unicodecsv as csv
import mailbox
import re
import sys
import talon
from talon import quotations
from BeautifulSoup import BeautifulSoup
from email.header import decode_header

talon.init()
stdout = sys.stdout
reload(sys)

sys.setdefaultencoding("ISO-8859-1")
# sys.setdefaultencoding("cp1252")
sys.stdout = stdout

In [27]:
def clean_content(content):
    # Basic Processing
    content = u'' + content
#     content = quotations.extract_from_plain(content) # Remove Signature Sections

    # Remove all lines below a section of the thread that starts with --. This is a naive approach but captures 
    # most cases we are interested in
    sign_index = content.find('--')
    if sign_index != -1:
        content = content[:sign_index]
    
    content = re.sub(r'[\w\.-]+@[\w\.-]+', ' ', content, re.MULTILINE) # Remove E-Mail Addresses
    content = BeautifulSoup(content).getText(separator=u' ') # Remove HTML        
#     content = re.sub(r'https?:\/\/.*[\r\n\"<>()\s\/$]', ' ', content, re.MULTILINE) # Remove Links URIs Etc                

    return (u'' + content.strip()).encode('utf-8')

def get_message_body(message):
    message_body = ""
    if message.is_multipart():
        for part in message.get_payload():
            part = part.get_payload(decode=True)
            if part:
                message_body += part
    else:
        message_body = message.get_payload(decode=True)
    return clean_content(message_body)


def decode_subject(subject):
    if "=?" in subject:
        result = ''
        for part in decode_header(subject):
            result += part[0] + ' '
        subject = u'' + result
    
    if subject == ' ' or subject == '':
        return u'(no subject)'
    return subject.strip()

def print_full(string):
    print([c for c in string])

In [28]:
mbox = mailbox.mbox('data/dbpedia-discussion-archive')
csv_writer = csv.writer(open('data/dbpedia-discussion-archive-no-sig.csv', 'wb'))

for message in mbox:    
    processed_msg = get_message_body(message)
    decode_subject(message['Subject'])
    csv_writer.writerow([message['Date'], message['Message-ID'], message['From'], decode_subject(message['Subject']), processed_msg])
    

In [143]:
with open('data/untitled.txt', 'r') as myfile:
    content = myfile.read()

content = re.sub(r'<\w+.*>.*<\/\w+>', ' ', content, re.MULTILINE) # Remove HTML
print content

<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 Transitional//EN"" "" <html xmlns="" <head>
 12814 <meta http-equiv=""Content-Type"" content=""text/html; charset=UTF-8"" />
 12815 
 12816 </head>
 12817 
 12818 <body>
 12819 <embed width=""488"" height=""366"" flashvars=""&displayheight=366&file=
 12820 <embed width=""488"" height=""274"" flashvars=""&displayheight=274&file=
 12821 <embed width=""488"" height=""274"" flashvars=""&displayheight=274&file=
 12822 
 12823 
 12824 
 12825 </body>
 12826 </html>
